In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,X):
        return X  - X.mean()


In [2]:
layer = CenteredLayer()

# 测试
layer(torch.FloatTensor([1,2,3,4,5]))

#  将层作为组件组合到更加复杂的模型中
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

tensor([-2., -1.,  0.,  1.,  2.])

In [3]:
# 定义自带参数的层  输入和输出  
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        # 初始化权重参数
        self.weight = nn.Parameter(torch.randn(in_units,units))
        # 初始化偏置
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self,X):
        #  计算线性层  X是输入的张量 self.weight.data是权重参数
        linear = torch.matmul(X,self.weight.data) + self.bias.data
        return F.relu(linear)
    

linear = MyLinear(5,3)
print(linear.weight)

Parameter containing:
tensor([[-0.1338,  1.6028,  0.2657],
        [ 0.3460,  0.5178,  1.2418],
        [-0.1137, -1.9221, -0.4234],
        [-0.0685,  0.6574, -1.1261],
        [-0.6649,  0.5456,  1.5100]], requires_grad=True)


In [4]:
linear(torch.randn(2,5))  # 两个样本 五个特征

tensor([[0.0000, 0.0000, 0.0000],
        [0.5056, 4.5267, 0.0000]])

In [5]:
# 使用自定义岑构建模型 
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))

net(torch.rand(2,64))

tensor([[18.2361],
        [17.0688]])

In [6]:
import torch
from torch import nn
from torch.nn import functional as F

# 创建一个长度为4的张量
x = torch.arange(4)
torch.save(x, 'x-file')

In [7]:
x2 = torch.load('x-file')
print(x2)

tensor([0, 1, 2, 3])


In [8]:
y = torch.zeros(4)
torch.save([x,y],'x-files')
x2,y2 = torch.load('x-files')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [10]:
# 创建张量字典  保存张量
mydict = {'x':x,'y':y}
torch.save(mydict,'mydict')

mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)

    def forward(self,X):
        return self.output(F.relu(self.hidden(X)))
    

net = MLP()
X = torch.randn(size= (2,20))
Y = net(X)

In [15]:
# 取出模型的参数保存在一个mlp.params文件中
torch.save(net.state_dict(),'mlp.params')

In [16]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [17]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [22]:
# 使用cpu
print(torch.device('cpu'))

# 使用gpu
print(torch.device(type='cuda'))

# 表示使用第1块gpu
print(torch.device('cuda:1'))

# 查询可用gpu的数量
torch.cuda.device_count()

cpu
cuda
cuda:1


0

In [23]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

In [25]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [30]:
X = torch.ones(2,3,device=try_gpu())
X.device
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [31]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.5478, 0.3206, 0.8756],
        [0.9405, 0.1232, 0.5788]])

In [32]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())  # 设置gpu

net(X)

tensor([[1.1469],
        [1.1469]], grad_fn=<AddmmBackward0>)

In [33]:
net[0].weight.data.device

device(type='cpu')